# Store Sales EDA and Analysis with Plotly

![corporacion_favorita.png](https://media-exp1.licdn.com/dms/image/C4E1BAQGfedhxSLXx0A/company-background_10000/0/1562626247545?e=2147483647&v=beta&t=sB7x6d7rdXQr_R3JixeHw8VxtpwA1TS--5XnUHteYVQ)

Dataset found at https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data.

We will be looking at the Store Sales dataset and looking for trends and insights into the business and eventually creating a dashboard in PowerBI using the information we retreive.


In this excercise we're taking a trip down to Equador where the director of Corporación Favorita has asked us to take a look at some of his data and come up with some analysis that can shed more light into his business

The data is provided to us by Favorita stores located in Ecuador. The training data includes dates, store and product information, whether that item was being promoted, as well as the sales numbers. 

The accompanying PowerBI dashboard can be found at [this link](https://www.dropbox.com/s/jat7hn9z0e2b7vk/super_store_dash.pbix?dl=0)

# <b> </b> <b style='color:red'>Contents</b><hr>

[**1.Understanding the Data**](#understanding)

[**2.Analysis**](#analysis)


## Importing the Data

In [ ]:
import pandas as pd
import numpy as np
import calendar
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.offline as offline
import plotly.graph_objs as go
offline.init_notebook_mode(connected = True)



In [ ]:
df_holiday = pd.read_csv("../input/store-sales-time-series-forecasting/holidays_events.csv")
df_oil = pd.read_csv("../input/store-sales-time-series-forecasting/oil.csv")
df_stores = pd.read_csv("../input/store-sales-time-series-forecasting/stores.csv")
df_test = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv")
df_train = pd.read_csv("../input/store-sales-time-series-forecasting/train.csv")
df_transactions = pd.read_csv("../input/store-sales-time-series-forecasting/transactions.csv")

<h1 style='color:red'><b>
<b id="understanding">Understanding the Data</b></h1><hr>

Let's look at each dataset individually to see what they tell us

In [ ]:
# First off let's create a function that will allow us to check everything in one go
def check_data(df):
    print(df.head())
    print(df.info())
    print(df.shape)

In [ ]:
# First the holiday data

check_data(df_holiday)

Looks like a list of holidays along with their dates and location.

Also of note the Favorita Stores team (in this case the description page of the dataset) tells us that:

* "NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge."

In [ ]:
# Now the oil data

check_data(df_oil)

Seems like the price of oil for each specific date

In [ ]:
# Now the stores data

check_data(df_stores)

Looks like the store number, along with the city and state as well as the type of store. We also see a cluster here and there and we can't figure out what that is so we ask the Favorita stores team what this means and they tell us that "cluster is a grouping of similar stores".

In [ ]:
# Now the train data

check_data(df_train)

Looks like we got a the number of the store, date of the purchase, what category the product falls into, the sales for that category on the day and whether or not it was on promotion.

In [ ]:
# Now for the test data

check_data(df_test)

Same thing as the train data except lacking the sales column

In [ ]:
# And now lastly the transaction

check_data(df_transactions)

Looks like we got a date, store number and how many transactions were made that day

The good news is that it looks like we don't have any missing values, however it does seem like some of the Dtypes are incorrect so we have to correct that.

In [ ]:
# Change the types

df_holiday["type"]=df_holiday["type"].astype("category")
df_holiday["date"]=pd.to_datetime(df_holiday["date"])

df_oil["date"]=pd.to_datetime(df_oil["date"])
df_oil.rename(columns={'dcoilwtico':'oil_price'}, inplace=True)

df_stores["cluster"]=df_stores.cluster.astype("category")
df_stores["type"]=df_stores.type.astype("category")


df_train.family=df_train.family.astype('category')
df_train.store_nbr= df_train.store_nbr.astype('category')
df_train.date=pd.to_datetime(df_train["date"])

df_transactions["date"]=pd.to_datetime(df_transactions["date"])

# Check it

def check_info(df):
    print(df.info())

check_info(df_holiday)
check_info(df_oil)
check_info(df_stores)
check_info(df_train)
check_info(df_transactions)


Now let's merge everything into one big dataframe

In [ ]:
# Merge the dataframes

df_merged =df_train.merge(df_holiday, on = 'date', how='left')
df_merged = df_merged.merge(df_oil, on = 'date', how='left')
df_merged = df_merged.merge(df_stores, on = 'store_nbr', how='left')
df_merged = df_merged.merge(df_transactions, on = ['date', 'store_nbr'], how='left')
df_merged = df_merged.rename(columns = {"type_x" : "holiday_type", "type_y" : "store_type"})

# Create new columns representing the year, month, week, day, quarter and day of the week

df_merged['date'] = pd.to_datetime(df_merged['date'])
df_merged['year'] = df_merged['date'].dt.year
df_merged['month'] = df_merged['date'].dt.month
df_merged['week'] = df_merged['date'].dt.isocalendar().week
df_merged['day'] = df_merged['date'].dt.day
df_merged['quarter'] = df_merged['date'].dt.quarter
df_merged['day_of_week'] = df_merged['date'].dt.day_name()


In [ ]:
# Save the new merged dataframe to a csv file for later use in PowerBI

# df_merged.to_csv("df_merged.csv")


<h1 style='color:red'><b>
<b id="analysis">Analysis</b></h1><hr> 

In [ ]:
# df_merged = pd.read_csv("df_merged.csv", low_memory= False)

In [ ]:
df_merged.info()

Now let's take a look at average sales by year, month, day of the week and quarter.

In [ ]:

# Making a dataframe grouping the year and sales in order to plot it soon
df_y_s = df_merged.groupby('year').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)
df_y_s['sales'] = round(df_y_s['sales'],2)

# Now creating one for the days of the week
df_dw_s = df_merged.groupby('day_of_week').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)
df_dw_s['sales'] = round(df_dw_s['sales'],2)

# For months
df_m_s = df_merged.groupby('month').agg({"sales" : "mean"}).reset_index()
df_m_s['sales'] = round(df_m_s['sales'],2)
df_m_s['month'] =['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

# Quarters
df_q_s = df_merged.groupby('quarter').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)[:10]
df_q_s['sales'] = round(df_q_s['sales'],2)

# Setting the colors I'm going to use
df_y_s['color'] =["#458cff","#63a7ff", "#82c2ff","#82c2ff","#a0dcff"]
df_m_s['color'] = ["#00876c","#3d9a70","#64ad73","#89bf77","#afd17c","#d6e184","#fff18f","#fdd576","#fbb862","#f59b56","#ee7d4f","#e35e4e"]
df_dw_s['color'] = ["#d43d51","#e6785c","#f3aa79", "#ffd9a6","#f4cc93","#e8c181","#dcb56e"]

# Creating subplots so we can have many graphs in the same image
fig = make_subplots(rows=2, cols=2, 
                    specs=[[{"type": "bar"},{"type": "bar"} ],
                           [{"type": "bar"}, {"type": "pie"}]],
                    subplot_titles=("Avg Sales by Year", "Avg Sales by Day of the Week", "Avg Sales by Month", "Avg Sales by Quarter"))
# Adding the charts I want to analyze
fig.add_trace(go.Pie(values=df_q_s['sales'], labels=df_q_s['quarter'], name='Quarter',
                     marker=dict(colors=["#0e455d", "#4f647d", "#8a8ba8", "#764c05"], line=dict(color='#000000', width=2)), hole=0,
                     hoverinfo='label+percent+value', textinfo='label'), 
                    row=2, col=2)
fig.update_traces(hovertemplate=None, textposition='outside',
                  textinfo='percent+label', rotation=0)
fig.append_trace(go.Bar(x=df_y_s['year'], y=df_y_s['sales'],textposition='auto',text=df_y_s['sales'], marker = {'color': list(df_y_s['color'])}),
                row=1, col=1)
fig.add_trace(go.Bar(x=df_m_s['month'], y=df_m_s['sales'], marker=dict(color= df_m_s['color']),
                     text=df_m_s['sales'],textposition='auto',
                     name='Month'), 
                     row=2, col=1)
fig.append_trace(go.Bar(x=df_dw_s['day_of_week'], y=df_dw_s['sales'], textposition='auto',text=df_dw_s['sales'],marker = {'color': list(df_dw_s['color'])}), row=1, col=2)


fig.update_layout(height=1000, width=1400, title_text="AVG SALES",  
                  title_font=dict(size=40, color='#8a8d93'), showlegend=False)

fig.show()




From this data we can see that:

* Weekends are unsurprisingly the days where most of sales occur
* Favorita stores have comparable sales month-by-month with the exception of the holiday month of December where the sales are much higher than usual.
* Sales are higher during the 4th quarter
* Favorita has enjoyed a year-by-year increase in sales with 2017 more than doubling the initial year of 2013

Now let's check out the state and city sales. We'll also check out Product Family information as well as Oil information

In [ ]:

# Creating a dataframe with the city and sales
df_c_s = df_merged.groupby('city').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)
df_c_s['sales'] = round(df_c_s['sales'],2)

# Now for the state
df_st_s = df_merged.groupby('state').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)
df_st_s['sales'] = round(df_st_s['sales'],2)

# Product family
df_f_s = df_merged.groupby('family').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)[:10]
df_f_s['sales'] = round(df_f_s['sales'],2)

# For oil in order to see if oil price affects sales or not
df_o_s = df_merged.groupby('oil_price').agg({"sales" : "mean"}).reset_index()
df_o_s['sales'] = round(df_o_s['sales'],2)

sizeref = 2.*max(df_o_s['sales'])/(30**2)

# Colors for the graphs
df_c_s['color'] = ["#440154","#471365","#482374","#46327e","#424086","#3d4e8a","#365c8d","#31688e","#2c738e","#277f8e","#238a8d","#1f968b","#1fa187","#26ad81","#35b779","#4ac16d","#65cb5e","#81d34d","#a0da39","#c0df25","#dfe318","#fde725"]
df_st_s['color'] = ["#002051","#022c65","#14386d","#2b446e","#42506e","#575c6e","#696970","#787573","#868276","#948f78","#a49d78","#b6ab73","#caba6a","#e0c95d","#f2d950","#fdea45"]
df_f_s['color'] = ["#0b090a","#161a1d","#660708","#a4161a","#ba181b","#e5383b","#b1a7a6","#d3d3d3","#f5f3f4","#ffffff"]

fig = make_subplots(rows=2, cols=2, 
                    specs=[[{"type": "bar"},{"type": "bar"} ],
                           [{"type": "bar"}, {"type": "scatter"}]],
                    subplot_titles=("Avg Sales by State", "Avg Sales by City", "Avg Sales by Product Family", "Sales by Oil Price"))


fig.append_trace(go.Bar(x=df_st_s['state'], y=df_st_s['sales'],textposition='auto',text=df_st_s['sales'], marker = {'color': list(df_st_s['color'])}),
                row=1, col=1)
fig.append_trace(go.Bar(x=df_c_s['city'], y=df_c_s['sales'],textposition='auto',text=df_c_s['sales'], marker = {'color': list(df_c_s['color'])}),
                row=1, col=2)
fig.append_trace(go.Bar(x=df_f_s['family'], y=df_f_s['sales'],textposition='auto',text=df_f_s['sales'], marker = {'color': list(df_f_s['color'])}),
                row=2, col=1)
fig.append_trace(go.Scatter(x = df_o_s['oil_price'], y = df_o_s['sales'],  mode = 'markers' , hovertemplate="<b>Oil_Price: %{x}<br><br><br>" + "<b>Sales: %{y}<br><br><br>", marker = dict(color = df_o_s['sales'],
                          colorscale = 'portland',opacity = 0.8,
                          sizemode = 'area', sizeref= sizeref,
                          sizemin= 4, showscale = False)), row=2, col=2)

fig.update_xaxes(title_text="Oil_Price", row=2, col=2)
fig.update_yaxes(title_text="Sales", row=2, col=2)
fig.update_layout(height=1000, width=1400, title_text="AVG SALES 2",  
                  title_font=dict(size=40, color='#8a8d93'), showlegend=False)

fig.show()

* Quito is the city with the most average sales which is unsurprising seeing as it's the capital and the second largest city in Equador. What is surprising though is that Guayaquil, the city with the largest population, is only 10th in the list. Meanwhile, Cayambe, a city 73 times smaller than Guayaquil is second.
 - **One thing to note is that this is the average and not total. So on average, people in Cayambe buy more than on Guayaquil but when we look at the total overall I expect Guayaquil to be higher**

 * The state of Pichincha (where Quito is a part of) has the most average sales

 * Grocery is the most bought items by far with beverages and produce coming in second

 * Looks like Oil price does influence sales as the higher it gets the lower the sales


In [ ]:
# Create a dataframe for city sales to assist in our plotting
df_cs_s = df_merged.groupby('city').agg({"sales" : "sum"}).reset_index().sort_values(by='sales', ascending=False)
df_cs_s['sales'] = round(df_cs_s['sales'],2)

# Now for the stores
df_store_s = df_merged.groupby(['city', 'store_type']).agg({"store_nbr" : "nunique"}).reset_index().sort_values(by='store_nbr', ascending=False)

# Now for the promotions
df_op_s = df_merged.groupby(['city']).agg(sales_mean = ('sales', 'mean'), onpromotion_mean = ('onpromotion', 'mean')).reset_index().sort_values(by='sales_mean', ascending=False)
df_op_s['sales_mean'] = round(df_op_s['sales_mean'],2)
sizeref2 = 2.*max(df_op_s['sales_mean'])*100


fig = make_subplots(rows=2, cols=2, 
                    specs=[[{"type": "bar"},{"type": "bar"} ],
                           [{"type": "bar"}, {"type": "scatter"}]],
                    subplot_titles=("Total Sales by State", "Total stores in City", "Average Items on Promotion"))


fig.append_trace(go.Bar(x=df_cs_s['city'], y=df_cs_s['sales'],textposition='auto',text=df_cs_s['sales'], marker = {'color': list(df_c_s['color'])}),
                row=1, col=1)
fig.append_trace(go.Bar(x=df_store_s['city'], y=df_store_s['store_nbr'],textposition='auto',text=df_store_s['store_nbr'], marker = {'color': list(df_st_s['color'])}),
                row=1, col=2)
fig.append_trace(go.Scatter(x = df_op_s['sales_mean'], y = df_op_s['onpromotion_mean'],  mode = 'markers+text' , text = df_op_s['city'],textposition='top center', marker = dict(color = df_op_s['sales_mean'],
                          colorscale = 'YlOrRd',opacity = 0.8,
                          sizemode = 'area', size = df_op_s['sales_mean'],
                          sizemin= 4, showscale = False)), row=2, col=1)
fig.update_layout(template="ggplot2",height=1000, width=1400, title_text="Total Sales by State, Store numbers and Average Items on Promotion",  
                  title_font=dict(size=40, color='#8a8d93'), showlegend=False)


fig.show()

* We can see that indeed Guayaquil has more total sales than Cayambe but it is still quite underwhelming when looking at the big picture.
* We also see that Guayaquil is the second city with the most stores with a total of 8 but still 10 less stores than Quito which has 18 in total
* We see that Cayambe has the highest average of items in promotion which probably help it become the second highest in average sales

<h1 style='color:red'><b>
<b id="end">End</b></h1><hr> 
    
Thank you for reading this analysis, please check out the accompanying PowerBI dashboard that can be found at [this link](https://www.dropbox.com/s/jat7hn9z0e2b7vk/super_store_dash.pbix?dl=0)